In [1]:
from __future__ import annotations

import numpy as np
from numpy import float32, float64, int32, int64, ndarray
from numpy.typing import NDArray
from PIL import ImageOps
import PIL.Image
from PIL.Image import Image, Resampling
from dataclasses import dataclass
from typing import Any, Optional, Protocol, Sequence, runtime_checkable, List, cast
from onnxruntime import InferenceSession
import os
import pandas as pd
import cv2

In [2]:

class PNodeArg(Protocol):
    @property
    def name(self) -> str:
        ...

    @property
    def shape(self) -> Any:
        ...

    @property
    def type(self) -> str:
        ...


class PSparseTensor(Protocol):
    values: ndarray
    indices: ndarray
    shape: tuple[int]

    @property
    def dtype(self) -> Any:
        ...

class PInferenceSession(Protocol):
    def run(
        self, output_names, input_feed: dict[str, Any], run_options=None
    ) -> list[ndarray] | list[list] | list[dict] | list[PSparseTensor]:
        ...

    def get_inputs(self) -> list[PNodeArg]:
        ...

In [3]:
@dataclass
class ImageTensor:
    original_size: tuple[int, int]
    scale_size: tuple[int, int]
    data: ndarray
    

@dataclass
class Label:
    x: int
    y: int
    width: int
    height: int
    classifier: str


@dataclass
class LabelImage:
    source: Optional[str]
    # path: str
    width: int
    height: int
    labels: Sequence[Label]

In [4]:
def compute_iou(box: NDArray[int32], boxes: NDArray[int32]) -> NDArray[float64]:
    # Compute xmin, ymin, xmax, ymax for both boxes
    xmin = np.minimum(box[0], boxes[:, 0])
    ymin = np.minimum(box[1], boxes[:, 1])
    xmax = np.maximum(box[0] + box[2], boxes[:, 0] + boxes[:, 2])
    ymax = np.maximum(box[1] + box[3], boxes[:, 1] + boxes[:, 3])

    # Compute intersection area
    intersection_area = np.maximum(0, xmax - xmin) * np.maximum(0, ymax - ymin)

    # Compute union area
    box_area = box[2] * box[3]
    boxes_area = boxes[:, 2] * boxes[:, 3]
    union_area = box_area + boxes_area - intersection_area

    # Compute IoU
    iou = intersection_area / union_area

    return iou



In [5]:
def nms(
    boxes: NDArray[int32], scores: NDArray[float32], iou_threshold: float
) -> list[int64]:
    # Sort by score
    sorted_indices = np.argsort(scores)[::-1]

    keep_boxes = []
    while sorted_indices.size > 0:
        # Pick the last box
        box_id = sorted_indices[0]
        keep_boxes.append(box_id)

        # Compute IoU of the picked box with the rest
        ious = compute_iou(boxes[box_id, :], boxes[sorted_indices[1:], :])

        # Remove boxes with IoU over the threshold
        keep_indices = np.where(ious < iou_threshold)[0]

        # print(keep_indices.shape, sorted_indices.shape)
        sorted_indices = sorted_indices[keep_indices + 1]

    return keep_boxes

In [6]:
def image_to_tensor(img: Image, model: PInferenceSession) -> ImageTensor:
    _, _, width, height = model.get_inputs()[0].shape

    img = ImageOps.exif_transpose(img)
    original_size = img.size

    img = ImageOps.contain(img, (width, height), Resampling.BILINEAR)
    scale_size = img.size

    img = ImageOps.pad(
        img, (width, height), Resampling.BILINEAR, (114, 114, 114), (0, 0)
    )
    data = np.array(img)

    data = data / 255.0
    data = data.transpose(2, 0, 1)
    tensor = data[np.newaxis, :, :, :].astype(np.float32)

    return ImageTensor(original_size, scale_size, tensor)

In [7]:
class Predictor:
    def __init__(
        self,
        model: PInferenceSession,
        names: list[str],
        conf_threshold: float = 0.25,
        iou_threshold: float = 0.7,
    ) -> None:
        self.__model = model
        self.__names = names
        self.__conf_threshold = conf_threshold
        self.__iou_threshold = iou_threshold

    def predict(self, img: Image | str) -> LabelImage:
        if isinstance(img, str):
            img = PIL.Image.open(img)

        tensor = image_to_tensor(img, self.__model)
        results = cast(List[ndarray], self.__model.run(None, {"images": tensor.data}))
        predictions = np.squeeze(results[0]).T

        scores = np.max(predictions[:, 4:], axis=1)
        keep = scores > self.__conf_threshold
        predictions = predictions[keep, :]
        scores = scores[keep]
        class_ids = np.argmax(predictions[:, 4:], axis=1)

        boxes = predictions[:, :4]
        boxes[:, 0:2] -= boxes[:, 2:4] / 2
        boxes /= np.array(
            [*tensor.scale_size, *tensor.scale_size], dtype=np.float32
        )
        boxes *= np.array([*tensor.original_size, *tensor.original_size])
        boxes = boxes.astype(np.int32)

        keep = nms(boxes, scores, self.__iou_threshold)
        labels = []
        for bbox, label in zip(boxes[keep], class_ids[keep]):
            labels.append(
                Label(
                    x=bbox[0].item(),
                    y=bbox[1].item(),
                    width=bbox[2].item(),
                    height=bbox[3].item(),
                    classifier=self.__names[label],
                )
            )

        img_width, img_height = img.size
        return LabelImage(
            source=None,
            # path=img.filename,  # type: ignore
            width=img_width,
            height=img_height,
            labels=labels,
        )

In [8]:
def read_names_file(file_path):
    with open(file_path, 'r') as file:
        objects = file.read().splitlines()
    return objects

In [9]:
# Code snippet to load the model and run the inference on a single image

def model_load(model , names_path):
    classes = read_names_file(names_path)
    session = InferenceSession(
        model,
        providers=[
            # "CUDAExecutionProvider",
            "CPUExecutionProvider",
        ],
    )
    predictor = Predictor(session, classes, conf_threshold = 0.3, iou_threshold = 0.4)
    return predictor

# Input the model path, .names file and the image path for inference
model = "optimized_model.onnx" 
names_path = "obj.names"
img_cv2 = cv2.imread("img.jpg")

img_rgb = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB)
img = PIL.Image.fromarray(img_rgb)

predictor = model_load(model, names_path)
results = predictor.predict(img)

print(results.labels)

[Label(x=1307, y=763, width=440, height=315, classifier='barcode'), Label(x=1298, y=2365, width=440, height=236, classifier='barcode'), Label(x=1260, y=1332, width=520, height=304, classifier='barcode'), Label(x=1334, y=1912, width=480, height=271, classifier='barcode'), Label(x=1057, y=3286, width=326, height=180, classifier='barcode'), Label(x=553, y=2220, width=92, height=87, classifier='qrcode'), Label(x=1114, y=2893, width=193, height=98, classifier='barcode'), Label(x=798, y=2230, width=94, height=85, classifier='qrcode'), Label(x=687, y=1201, width=108, height=111, classifier='qrcode'), Label(x=803, y=1782, width=100, height=98, classifier='qrcode'), Label(x=535, y=1773, width=100, height=102, classifier='qrcode'), Label(x=667, y=1776, width=106, height=103, classifier='qrcode'), Label(x=673, y=2223, width=97, height=89, classifier='qrcode'), Label(x=544, y=1204, width=111, height=115, classifier='qrcode'), Label(x=855, y=731, width=409, height=115, classifier='barcode'), Label(

In [10]:
# Code snippet for testing the onnx model
def onnx_testing(predictor, annotated_label_dir, image_base_dir):
    image_dir_list = os.listdir(image_base_dir)
    result_list = []
    for base_dir in image_dir_list:
        
        if base_dir == '.DS_Store': continue
    
        image_dir = image_base_dir + base_dir + '/'
        image_list = os.listdir(image_dir)
    
        for count, images in enumerate(image_list):
            result_dict = {}
            result_dict["Image Name"] = images
        
            barCode = 0
            qrCode = 0
            predBarCode = 0
            predQrCode = 0
            ext = images.split(".")[-1]
            if ext != 'jpg': continue
            image_path = image_dir + images
        
            if os.path.exists(image_path):
                image_cv2 = cv2.imread(image_path)
                image_rgb = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
                image = PIL.Image.fromarray(image_rgb)
                results = predictor.predict(image)
                classes = []
                boxes = []
                for i in range(0 , len(results.labels)):
                    classes.append(results.labels[i].classifier)
                    x = results.labels[i].x
                    y = results.labels[i].y
                    width = results.labels[i].width
                    height = results.labels[i].height
                    box_coord = (x , y , width , height) 
                    boxes.append(box_coord)
        
                for items in classes:
                    if items == "qrcode":
                        predQrCode+=1
                    if items == "barcode":
                        predBarCode+=1
            
                print(f'Succesfully processed {count+1} images', end='\r')

                annot_label = images.replace(".jpg", ".txt")
                annot_label_path = annotated_label_dir + annot_label


                if os.path.exists(annot_label_path):
                    with open(annot_label_path, 'r') as file:
                        data_list = file.readlines()
                    for data in data_list:
                        pred_class, cx, cy, w, h = data.split(" ")
                        if pred_class == '1':
                            qrCode+=1
                        if pred_class == '0':
                            barCode+=1
                    result_dict["Actual QR"] = qrCode
                    result_dict["Predicted QR"] = predQrCode
                    result_dict["Actual Barcode"] = barCode
                    result_dict["Predicted Barcode"] = predBarCode

                if not os.path.exists(annot_label_path):
                    result_dict["Actual QR"] = 'NA'
                    result_dict["Predicted QR"] = predQrCode
                    result_dict["Actual Barcode"] = 'NA'
                    result_dict["Predicted Barcode"] = predBarCode
                result_dict["Bounding Boxes"] = boxes
            result_list.append(result_dict)
        print('\n')
        print(f'Sucessfully processed {base_dir} dir', end='\r')
        
    return result_list

# Input the annotated label directory and image folder path
annot_label_dir = 'labels/labels/'
img_base_dir = 'Chroma_Image_Analytics_2909/Chroma_Image_Analytics_2909/'

result_list = onnx_testing(predictor, annot_label_dir, img_base_dir)


C:\Users\dhava\AppData\Local\Temp\ipykernel_2244\3363175807.py:17: RuntimeWarning: divide by zero encountered in divide
  iou = intersection_area / union_area


Succesfully processed 197 images

Succesfully processed 86 imagestion KYBM and Accessories dir

Succesfully processed 30 images KYBM and Accesorries dir

Succesfully processed 61 imageseadphone,Soundbar,Speaker dir



In [11]:
df = pd.DataFrame(result_list)
df

,Image Name,Actual QR,Predicted QR,Actual Barcode,Predicted Barcode,Bounding Boxes
0,IMG20230105115725.jpg,0,0,1,1,"[(1012, 3401, 366, 116)]"
1,IMG20230105120217.jpg,0,0,6,6,"[(1847, 2619, 411, 224), (1878, 2225, 416, 205..."
2,IMG20230105120254.jpg,1,0,1,2,"[(1813, 3573, 645, 330), (349, 2744, 575, 559)]"
3,IMG20230105120349.jpg,1,1,4,4,"[(992, 1003, 359, 362), (1016, 2026, 412, 197)..."
4,IMG20230105120349_01.jpg,1,1,4,4,"[(1723, 2732, 356, 358), (1572, 1397, 405, 195..."
...,...,...,...,...,...,...
227,IMG20230105141539.jpg,1,1,0,0,"[(1288, 683, 178, 193)]"
228,IMG20230105141544.jpg,1,1,0,0,"[(1363, 1821, 144, 142)]"
229,IMG20230105141549.jpg,1,1,0,0,"[(1489, 698, 164, 171)]"
230,Scan from 2023-01-05 03_51_53 PM.jpg,0,0,2,2,"[(0, 794, 261, 216), (521, 1314, 389, 83)]"


In [12]:
len(result_list)

232

In [13]:
# Input the out directory
out_dir = 'test_results/'
excel_path = out_dir + 'model_metrics.xlsx'
df.to_excel(excel_path)